# Magic board

Create your own magic board to draw in the air with your finger by using hand tracking detection.

In [3]:
# Importation of libraries.
import cv2
import mediapipe as mp
import numpy as np

# Colors to paint.
colorCeleste = (255, 113, 82)
colorAmarillo = (89, 222, 255)
colorRosa = (128, 0, 255)
colorVerde = (0, 255, 36)
colorLimpiarPantalla = (29, 112, 246)
color = colorCeleste

# Thickness of the lines of the upper-left boxes which indicate the colors to draw.
grosor = 3
grosorCeleste = 6
grosorAmarillo = 2
grosorRosa = 2
grosorVerde = 2

# Thickness of the lines of the upper-right boxes which indicate the thickness of the marker to draw.
grosorPeque = 6
grosorMedio = 1
grosorGrande = 1

# Initialize variables of coordinates.
x1 = None
y1 = None
# This variable will create a black image that will help ups to draw later on.
imAux = None

# To draw the 21 points and its connections.
mp_drawing = mp.solutions.drawing_utils 
# Mediapipe hand detection solution.
mp_hands = mp.solutions.hands

# Capture the video: 0 for the camera of your computer, 1 if it's external.
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)

# Set your configuration options.
with mp_hands.Hands(
    static_image_mode = False,
    max_num_hands=2,
    min_detection_confidence=0.5) as hands:
    
    while True:
        # The variable frame stores what the image is capturing.
        ret, frame = cap.read()
        if ret == False:
            break
        
        # Get size of the captured image.
        height, width, _ = frame.shape
        # Flip the image to avoid mirroring.
        frame = cv2.flip(frame, 1)
        # Change the image from BGR to RGB to make the detection.
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Create a matrix of zeros of the same size as frame that will acts as our board.
        if imAux is None: imAux = np.zeros(frame.shape, dtype=np.uint8)
        
        
        # Create color, thickness and clear screen boxes.
        
        cv2.rectangle(frame, (0,0), (50,50), colorAmarillo, grosorAmarillo)
        cv2.rectangle(frame, (50,0), (100,50), colorRosa, grosorRosa)
        cv2.rectangle(frame, (100,0), (150,50), colorVerde, grosorVerde)
        cv2.rectangle(frame, (150,0), (200,50), colorCeleste, grosorCeleste)

        cv2.rectangle(frame, (300,0), (400,50), colorLimpiarPantalla,1)
        cv2.putText(frame, "Limpiar",(320,20),6,0.6, colorLimpiarPantalla,1, cv2.LINE_AA)
        cv2.putText(frame, "Pantalla",(320,40),6,0.6, colorLimpiarPantalla,1, cv2.LINE_AA)

        cv2.rectangle(frame,(490,0),(540,50),(0,0,0),grosorPeque)
        cv2.circle(frame,(515,25),3,(0,0,0),-1)
        cv2.rectangle(frame,(540,0),(590,50),(0,0,0),grosorMedio)
        cv2.circle(frame,(565,25),7,(0,0,0),-1)
        cv2.rectangle(frame,(590,0),(640,50),(0,0,0),grosorGrande)
        cv2.circle(frame,(615,25),11,(0,0,0),-1)
        
        # Hands.process returns the hand landmarks.
        results = hands.process(frame_rgb)
        
        # Collection of detected/tracked hands, where each hand is represented as a list of 21 hand landmarks and each landmark is composed of x, y and z. 
        if results.multi_hand_landmarks is not None:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    frame, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(255,124,199), thickness = 1),
                    mp_drawing.DrawingSpec(color=(255,0,145), thickness = 1)
                    )

        # Get index and middle fingers tip and middle position to recognize if the fingers are up or down.    
        
            # IndexTip
            x_indext = int(hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * width)
            y_indext = int(hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * height)
            indext = (x_indext, y_indext)
            
            # IndexMiddle
            x_indexm = int(hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_PIP].x * width)
            y_indexm = int(hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_PIP].y * height)
            indexm = (x_indexm, y_indexm)

            # MiddleTip
            x_middlet = int(hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].x * width)
            y_middlet = int(hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].y * height)
            middlet = (x_middlet, y_middlet)
            
            # MiddleMiddle
            x_middlem = int(hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_PIP].x * width)
            y_middlem = int(hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_PIP].y * height)
            middlem = (x_middlem, y_middlem)
            
            # Initialize a variable to detect if you are in drawing or selecting mode.
            mode = 0
            
            # SELECT COLOR MODE - TWO FINGERS UP
            if indext > indexm and middlet > middlem:
                cv2.putText(frame,"Select Color Mode", (10,460), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2)
                mode = "selectcolor"
                
            # DRAWING MODE - ONE FINGER UP 
            if indext > indexm and middlet < middlem:
                cv2.putText(frame,"Drawing Mode", (10,460), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2)
                mode = "drawingmode"
            
            # Match finger and board coordinates.
            if mode == "drawingmode":
                x2 = x_indext
                y2 = y_indext
            
            # Give fingers the functionality to draw or select color/thickness.
                if x1 is not None:
                    
                    # Choose color to draw. 
                    if 0 < x2 < 50 and 0 < y2 < 50:
                        color = colorAmarillo
                        grosorAmarillo = 6
                        grosorRosa = 2
                        grosorVerde = 2
                        grosorCeleste = 2

                    if 50 < x2 < 100 and 0 < y2 < 50:
                        color = colorRosa
                        grosorAmarillo = 2
                        grosorRosa = 6
                        grosorVerde = 2
                        grosorCeleste = 2

                    if 100 < x2 < 150 and 0 < y2 < 50:
                        color = colorVerde
                        grosorAmarillo = 2
                        grosorRosa = 2
                        grosorVerde = 6
                        grosorCeleste = 2

                    if 150 < x2 < 200 and 0 < y2 < 50:
                        color = colorCeleste
                        grosorAmarillo = 2
                        grosorRosa = 2
                        grosorVerde = 2
                        grosorCeleste = 6
                        
                    # Choose thickness to draw.
                    if 490 < x2 < 540 and 0 < y2 < 50:
                        grosor = 3
                        grosorPeque = 6
                        grosorMedio = 1
                        grosorGrande = 1

                    if 540 < x2 < 590 and 0 < y2 < 50:
                        grosor = 7
                        grosorPeque = 1
                        grosorMedio = 6
                        grosorGrande = 1

                    if 590 < x2 < 640 and 0 < y2 < 50:
                        grosor = 11
                        grosorPeque = 1
                        grosorMedio = 1
                        grosorGrande = 6
                    
                    # Clear screen
                    if 300 < x2 < 400 and 0 < y2 < 50:
                        cv2.rectangle(frame, (300,0), (400,50), colorLimpiarPantalla,1)
                        cv2.putText(frame, "Limpiar",(320,20),6,0.6, colorLimpiarPantalla,2, cv2.LINE_AA)
                        cv2.putText(frame, "Pantalla",(320,40),6,0.6, colorLimpiarPantalla,2, cv2.LINE_AA)
                        imAux = np.zeros(frame.shape, dtype=np.uint8)
                    
                    # Draw
                    if 0 < y2 < 60 or 0 < y1 < 60:
                        imAux = imAux
                    else: 
                        imAux = cv2.line(imAux,(x1,y1),(x2,y2),color,grosor)
                cv2.circle(frame,(x2,y2),grosor,color,3)
                x1 = x_indext
                y1 = y_indext

            else:
                x1 = None
                y1 = None
        
        # Convert the board from BGR to Gray Scale.
        imAuxGray = cv2.cvtColor(imAux,cv2.COLOR_BGR2GRAY)
        # Obtain a binary image applying a threshold.
        _ , th = cv2.threshold(imAuxGray, 10, 255, cv2.THRESH_BINARY)
        thInv = cv2.bitwise_not(th)
        # Obtain a new image with the inverted mask.
        frame = cv2.bitwise_and(frame,frame, mask = thInv)
        # Add the image with the drawings to the VideoCapture.
        frame = cv2.add(frame,imAux)
        
        # Display an image window. 
        cv2.imshow("Magic Board", frame)      
        
        # Close the image window by pressing 'q' for 1 second.
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break   
            
cap.release()
cv2.destroyAllWindows()